# Calculate monthly mean reservoir inflows for reservoir segments from natural lakes simulation
postprocessing from Cheyenne - global simulations with HDMA topology

Inne Vanderkelen - March 2021

In [1]:
# import modules
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import os



# constants
secperday   = 86400
secperhour  = 3600
dt          = 86400 # time of the modeling can be also hourly for mizuRoute
daypermonth = 30 # average day in a month

In [2]:
# define directories

# model directory
outdir = '/glade/work/ivanderk/mizuRoute_global/route/'

# current working directory
scriptsdir = os.getcwd() + '/'

# give simulation name
casename= 'natlake_global'

# simulation years
start_year = 1985
end_year = 2000

# open topology 
ntopo = xr.open_dataset(outdir+'ancillary_data/ntopo_hdma_mod.reorder_lake_H06.nc')

In [4]:
# get reservoir pfafs
res_mask   = ntopo['lake_model']==2
res_seg_id = ntopo['seg_id'].where(res_mask, drop=True)

In [6]:
# laod all years and append into one dataset with storage, inflow and outflow
ds_res_year_all =  []

for year in range(start_year,end_year+1):
    # Open simulation for one year
    ds = xr.open_dataset(outdir+"output/"+casename+".mizuRoute.h."+str(year)+"-01-01-00000.nc")
    
    # extract inflow  for all reservoirs
    ds_res_year_all.append(ds['sumUpstreamRunoff'].where(res_mask, drop=True))

ds_res = xr.concat(ds_res_year_all, dim='time')

In [7]:
# caluclate seasonality
ds_res_monmean = ds_res.groupby('time.month').mean()


In [11]:
# save inflow seasonality in vsc
inflow_df = pd.DataFrame(index=list(ntopo['PFAF'].where(res_mask, drop=True).values))
inflow_df.index.name = 'PFAF'
inflow_df.index.astype('str')

months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i,month in enumerate(months):
    inflow_df['H06_I_'+month] = ds_res_monmean[i,:].values
    
inflow_df.to_csv(scriptsdir+'reservoirs_monthly_inflow.csv')

In [12]:
inflow_df

,H06_I_Jan,H06_I_Feb,H06_I_Mar,H06_I_Apr,H06_I_May,H06_I_Jun,H06_I_Jul,H06_I_Aug,H06_I_Sep,H06_I_Oct,H06_I_Nov,H06_I_Dec
PFAF,,,,,,,,,,,,
2524449,4.923365,10.747301,5.666023,2.253487,0.723062,0.547732,2.802952e-01,2.416368e-01,0.567333,1.620724,3.132857,3.941750
2398995,47.008736,51.285648,38.555130,12.927869,4.240016,1.193391,1.055577e+00,9.133393e-01,0.993154,5.127810,10.564802,21.726988
25335565,15.428672,18.242216,16.238842,9.320963,2.408906,2.861431,1.581339e+00,1.971171e+00,5.134433,9.849731,14.743700,16.114023
2924743,4.522735,4.217585,4.404770,5.368041,7.278252,9.425138,1.144888e+01,1.580988e+01,11.732697,6.505398,4.904685,5.034019
255087443,2.417278,2.904975,2.148941,0.351496,0.004040,0.000360,9.796076e-07,9.796076e-07,0.003419,0.115828,0.961233,1.481059
...,...,...,...,...,...,...,...,...,...,...,...,...
5741179,10156.558594,9131.584961,8504.842773,4304.181641,1639.295654,899.938599,7.134644e+02,6.247104e+02,1307.921021,3082.268066,7959.333984,11271.772461
5741171,10173.357422,9152.245117,8532.643555,4334.691895,1664.117920,932.949280,7.421839e+02,6.389658e+02,1313.525146,3086.302490,7970.140625,11294.480469
5613,21304.839844,24415.462891,27610.388672,21981.451172,12253.376953,7105.275879,5.289490e+03,4.396597e+03,5363.744629,9196.685547,13750.547852,19280.917969
